# Kappa App for nasa log view calculation

In [ ]:
from pyspark.sql import SparkSession

# Set the necessary variables
# The following links were used to determine the necessary packages to include:
# - https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html and 
# - https://github.com/OneCricketeer/docker-stacks/blob/master/hadoop-spark/spark-notebooks/kafka-sql.ipynb  

scala_version = '2.12'  
spark_version = '3.5.3'
bootstrap_servers = ['localhost:9092']
topic_name = 'kappa-topic'
window_duration = '1 minute'
sliding_duration = '1 minute'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.9.0',
    'org.mariadb.jdbc:mariadb-java-client:3.5.2'
]

spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .config("SQLConf.ADAPTIVE_EXECUTION_ENABLED.key", "false")\
   .getOrCreate()
#.config("spark.executor.extraClassPath", "/Users/sturm/Development/cloud_computing_big_data/lib/mariadb-java-client-3.5.2.jar")\

In [ ]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", *bootstrap_servers)\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()
print(kafkaDf.isStreaming)    # Returns True for DataFrames that have streaming sources
kafkaDf.printSchema()

## Structure the value column

In [ ]:
from pyspark.sql.functions import col, concat, lit, split, from_unixtime
#col_schema = ["line_no INTEGER","host STRING","time TIMESTAMP","method STRING","url STRING","response INTEGER","bytes INTEGER"]
#schema_str = ",".join(col_schema)
structured_df = kafkaDf.select(
    concat(col("topic"), lit(':'), col("partition").cast("string")).alias("topic_partition"),
    col("offset"),
    col("value").cast("string"),
    col("timestamp"),
    col("timestampType"),
    split(col("value").cast("string"),",").getItem(0).alias("line_no").cast("integer"),
    split(col("value").cast("string"),",").getItem(1).alias("host").cast("string"),
    from_unixtime(split(col("value").cast("string"),",").getItem(2)).alias("time").cast("timestamp"),
    split(col("value").cast("string"),",").getItem(3).alias("method").cast("string"),
    split(col("value").cast("string"),",").getItem(4).alias("url").cast("VARCHAR(255)"),
    split(col("value").cast("string"),",").getItem(5).alias("response").cast("integer"),
    split(col("value").cast("string"),",").getItem(6).alias("bytes").cast("integer"),   
    )

In [ ]:
from pyspark.sql.functions import window,endswith,when, coalesce

page_count_df = structured_df.filter(structured_df["method"] == "GET")\
     .filter(structured_df["url"].endswith(".html"))\
     .groupBy (window(
        col("time"),
        window_duration,
        sliding_duration
    ),
    col("url")).count() \
     .withColumnRenamed('window.start', 'window_start') \
     .withColumnRenamed('window.end', 'window_end')\
     .withColumn("url", coalesce(col("url"), lit("unknown")))
page_count_df.printSchema()

In [ ]:
# Write to MariaDB 
dbUrl = 'jdbc:mysql://localhost:3306/kappa-view?permitMysqlScheme' # Use of jdbc:mysql://HOST/DATABASE?permitMysqlScheme is MANDATORY
dbOptions = {"user": "root", "password": "example", "driver": "org.mariadb.jdbc.Driver", "truncate": "true"}
dbSchema = 'pagecount'

def saveToDatabase(batchDataframe, batchId):
    global dbUrl, dbSchema, dbOptions
    print(f"Writing batchID {batchId} to database @ {dbUrl}")
    batchDataframe.distinct().write.jdbc(dbUrl, dbSchema, "overwrite", dbOptions)


db_insert_stream = page_count_df \
   .select(col('url'), col('count')) \
   .writeStream \
   .outputMode("complete") \
   .foreachBatch(saveToDatabase) \
   .start()
db_insert_stream.awaitTermination()

#TODO continue: build a consistent view with one count for every url by extending the window or disable the window 

In [ ]:
spark.stop()